# About me

In [62]:
# Challenge for Fintual application 
# To give the challenge a plus, I use yfinance library to get some stock prices, interpolating the missing data

# Imports

In [63]:
from datetime import datetime
from typing import List
import yfinance as yf # using version 0.2.54
import pandas as pd
import random

yf.__version__

'0.2.54'

In [64]:
class Stock:
    
    def __init__(self, name: str, yf_name: str):
        self.name = name # Stock name
        self.yf_name = yf_name # Stock name for yfinance library
    
    def yf_download(self) -> pd.DataFrame:
        df = yf.download(self.yf_name) # download info from yfinance
        df = df.reset_index()
        df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d').dt.date # date format
        df.columns = ['DATE', 'Close', 'High', 'Low', 'Open', 'Volume'] # columns names
        df = df.rename(columns = {'Close': 'PRICE', 'Date': 'DATE'}) # rename column
        df = df[['DATE', 'PRICE']] # select columns
        
        #display(df)
        # using interpolatation to complete all dates
        df_dates = pd.DataFrame({'DATE': pd.date_range(start = df['DATE'].min(), end = df['DATE'].max())})
        df_dates['DATE'] = pd.to_datetime(df_dates['DATE'], format = '%Y-%m-%d').dt.date
        #display(df_dates)
        self.df_prices = pd.merge(df_dates, df, on = 'DATE', how = 'left')
        self.df_prices['PRICE'] = self.df_prices['PRICE'].interpolate()
        
        return None
    
    def get_price_dict(self): # return relation between dates and prices
        self.yf_download()
        self.prices_dict = dict(zip(self.df_prices['DATE'], self.df_prices['PRICE']))
        
        return None
    
    def get_price(self, date: str) -> float:
        date = datetime.strptime(date, "%Y-%m-%d").date() # date format 
        return self.prices_dict[date] # return price of stock in a date
        

class Portfolio:
    
    def __init__(self):
        self.stocks_list = [] # List of stocks
    
    def add_stock(self, stock: Stock, quantity: float) -> None: # stock object and quantity of stock
        self.stocks_list.append((stock, quantity)) # add stock to list with its quantity
    
    def get_profit(self, start_date: str, end_date: str, annualized: bool = False) -> float:
        ''' Get profit from portfolio between two dates '''
        ''' if annualized is True, return annualized profit '''
        
        initial_value = sum(stock.get_price(start_date) * quantity for stock, quantity in self.stocks_list) # initial value of portfolio
        final_value = sum(stock.get_price(end_date) * quantity for stock, quantity in self.stocks_list) # final value of portfolio

        profit = final_value / initial_value - 1  # Return

        if annualized:
            start = datetime.strptime(start_date, "%Y-%m-%d")
            end = datetime.strptime(end_date, "%Y-%m-%d")
            years = (end - start).days / 365.25 # Calculate of CAGR
            return (1 + profit) ** (1 / years) - 1  # Fórmula de retorno anualizado

        return profit

# Example

In [65]:
initial_date, final_date = '2020-01-01', '2025-03-01'

In [66]:
stocks_0 = {'Apple': 'AAPL', 'Microsoft': 'MSFT', 'Amazon': 'AMZN', 'Google': 'GOOGL', 'Bitcoin': 'BTC-USD'}

stocks_objects = {}
portfolio = Portfolio() # Initialize portfolio object

for name in stocks_0:
    stock = Stock(name, stocks_0[name])
    stock.get_price_dict()
    initial_price = stock.get_price(initial_date) # get initial price of stock
    
    stock_usd = 10 + 90 * random.random() # random usd between 10 and 100
    quantity = stock_usd / initial_price
    portfolio.add_stock(stock, quantity) # add stock
    
    print(f'Stock Name: {name} | Initial Price: {round(initial_price, 2)} | Quantity: {round(quantity, 2)} | Initial USD: {round(stock_usd, 2)}')
    
    stocks_objects[name] = stock # add stock object to dictionary
    
profit = portfolio.get_profit(initial_date, final_date, annualized = False) # get profit of portfolio
print(f'Profit: {round(100 * profit, 2) }%')
annualized_profit = portfolio.get_profit(initial_date, final_date, annualized = True) # get annualized profit of portfolio
print(f'Annualized Profit: {round(100 * annualized_profit, 2)} %')
    

[*********************100%***********************]  1 of 1 completed


Stock Name: Apple | Initial Price: 71.91 | Quantity: 1.04 | Initial USD: 74.65


[*********************100%***********************]  1 of 1 completed


Stock Name: Microsoft | Initial Price: 151.93 | Quantity: 0.41 | Initial USD: 61.82


[*********************100%***********************]  1 of 1 completed


Stock Name: Amazon | Initial Price: 93.65 | Quantity: 0.96 | Initial USD: 89.84


[*********************100%***********************]  1 of 1 completed


Stock Name: Google | Initial Price: 67.46 | Quantity: 1.42 | Initial USD: 95.69


[*********************100%***********************]  1 of 1 completed

Stock Name: Bitcoin | Initial Price: 7200.17 | Quantity: 0.0 | Initial USD: 31.5
Profit: 247.34%
Annualized Profit: 27.27 %
